## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bugama,NG,4.7341,6.8635,78.85,87,91,3.91,overcast clouds
1,1,Cabedelo,BR,-6.9811,-34.8339,77.67,82,59,11.39,broken clouds
2,2,Mataura,NZ,-46.1927,168.8643,58.48,71,12,1.50,few clouds
3,3,Newport,GB,51.5877,-2.9984,47.08,77,100,8.03,overcast clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,67.17,49,8,10.42,clear sky


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bugama,NG,4.7341,6.8635,78.85,87,91,3.91,overcast clouds
1,1,Cabedelo,BR,-6.9811,-34.8339,77.67,82,59,11.39,broken clouds
9,9,Bam,IR,29.1060,58.3570,77.32,19,0,9.22,clear sky
11,11,Caravelas,BR,-17.7125,-39.2481,75.31,85,0,5.35,clear sky
12,12,Rikitea,PF,-23.1203,-134.9692,77.38,64,21,18.39,few clouds
13,13,Arraial Do Cabo,BR,-22.9661,-42.0278,76.32,85,8,20.31,clear sky
16,16,Taoudenni,ML,22.6783,-3.9836,84.16,11,0,7.43,clear sky
18,18,Ilhabela,BR,-23.7781,-45.3581,78.31,73,50,4.68,scattered clouds
20,20,Faanui,PF,-16.4833,-151.7500,82.20,70,5,22.26,clear sky
23,23,Jizan,SA,17.3333,42.6667,86.94,59,2,4.00,clear sky


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bugama,NG,78.85,overcast clouds,4.7341,6.8635,
1,Cabedelo,BR,77.67,broken clouds,-6.9811,-34.8339,
9,Bam,IR,77.32,clear sky,29.1060,58.3570,
11,Caravelas,BR,75.31,clear sky,-17.7125,-39.2481,
12,Rikitea,PF,77.38,few clouds,-23.1203,-134.9692,
13,Arraial Do Cabo,BR,76.32,clear sky,-22.9661,-42.0278,
16,Taoudenni,ML,84.16,clear sky,22.6783,-3.9836,
18,Ilhabela,BR,78.31,scattered clouds,-23.7781,-45.3581,
20,Faanui,PF,82.20,clear sky,-16.4833,-151.7500,
23,Jizan,SA,86.94,clear sky,17.3333,42.6667,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping.")  
        

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [41]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bugama,NG,78.85,overcast clouds,4.7341,6.8635,Johnbull polo
1,Cabedelo,BR,77.67,broken clouds,-6.9811,-34.8339,Hotel Pigalle
9,Bam,IR,77.32,clear sky,29.1060,58.3570,Bam Tourist Hotel
11,Caravelas,BR,75.31,clear sky,-17.7125,-39.2481,Pousada dos Navegantes
12,Rikitea,PF,77.38,few clouds,-23.1203,-134.9692,People ThankYou
13,Arraial Do Cabo,BR,76.32,clear sky,-22.9661,-42.0278,Pousada Porto Praia
18,Ilhabela,BR,78.31,scattered clouds,-23.7781,-45.3581,Ilha Flat Hotel
20,Faanui,PF,82.20,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
23,Jizan,SA,86.94,clear sky,17.3333,42.6667,شهدان. ابوالحجر
27,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,Paradise Inn


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))